<a href="https://colab.research.google.com/github/lynguyenp/Hidden-Concept/blob/main/20250222_Final_Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ENVIRONMENT PREPARATION**

In [ ]:
!pip install nest_asyncio nltk pandas tqdm aiohttp datasets transformers torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**IMPORTING LIBRARIES**

In [ ]:
import pandas as pd
import aiohttp
import asyncio
import nltk
import torch.nn as nn
import torch
import seaborn as sns


from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from datasets import load_dataset


from transformers import AutoTokenizer, AutoModel, AdamW
from tqdm import tqdm

from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, TensorDataset, Dataset

from torch.optim import AdamW
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.cuda.amp import GradScaler, autocast
from multiprocessing import Pool

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline

from matplotlib import pyplot as plt


nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop_words = set(stopwords.words('english'))
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**LOADING DATA**

In [ ]:
train_relation = "/content/drive/MyDrive/3. MSc. Data Science & Economics/10. Text Mining & Sentiment Analysis/Project /Data/relation_conceptnet.csv"
train_relation = pd.read_csv(train_relation)
test_relation = "/content/drive/MyDrive/3. MSc. Data Science & Economics/10. Text Mining & Sentiment Analysis/Project /Data/test_relation_conceptnet.csv"
test_relation = pd.read_csv(test_relation)
dev_relation = "/content/drive/MyDrive/3. MSc. Data Science & Economics/10. Text Mining & Sentiment Analysis/Project /Data/dev_relation_conceptnet.csv"
dev_relation = pd.read_csv(dev_relation)


In [ ]:
dataset = load_dataset("snli")
snli_train = pd.DataFrame(dataset['train'])
snli_train = snli_train[snli_train['label'] != -1]
snli_train = snli_train.head(20000).copy()

snli_dev = pd.DataFrame(dataset['validation'])
snli_dev = snli_dev[snli_dev['label'] != -1]
snli_dev = snli_dev.head(2500).copy()

snli_test = pd.DataFrame(dataset['test'])
snli_test = snli_test[snli_test['label'] != -1]
snli_test = snli_test.head(2500).copy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

**I. PRE-TRAINED LARGE LANGUAGE MODELS**

1. Data Preprocessing

In [ ]:
# Prepare the data
def preprocess_data(tokenizer, df, max_length=128):
    encodings = tokenizer(list(df['premise']), list(df['hypothesis']), truncation=True, padding=True, max_length=max_length)
    labels = df['label'].values
    dataset = TensorDataset(torch.tensor(encodings['input_ids']), torch.tensor(encodings['attention_mask']), torch.tensor(labels))
    return dataset

In [ ]:
# Tokenizers for each model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# parallel processing
def tokenize_data(args):
    tokenizer, df = args
    return preprocess_data(tokenizer, df)

# arguments for parallel processing
tokenizers = [bert_tokenizer, roberta_tokenizer, distilbert_tokenizer]
datasets = [snli_train, snli_dev, snli_test]

# list of arguments
args = [(tokenizer, df) for tokenizer in tokenizers for df in datasets]

# tokenize in parallel
with Pool(processes=3) as pool:
    results = pool.map(tokenize_data, args)

# results
train_dataset_bert, dev_dataset_bert, test_dataset_bert = results[0:3]
train_dataset_roberta, dev_dataset_roberta, test_dataset_roberta = results[3:6]
train_dataset_distilbert, dev_dataset_distilbert, test_dataset_distilbert = results[6:9]

In [ ]:
# Data loaders
batch_size = 16
train_loader_bert = DataLoader(train_dataset_bert, batch_size=batch_size, shuffle=True)
dev_loader_bert = DataLoader(dev_dataset_bert, batch_size=batch_size)
test_loader_bert = DataLoader(test_dataset_bert, batch_size=batch_size)

train_loader_roberta = DataLoader(train_dataset_roberta, batch_size=batch_size, shuffle=True)
dev_loader_roberta = DataLoader(dev_dataset_roberta, batch_size=batch_size)
test_loader_roberta = DataLoader(test_dataset_roberta, batch_size=batch_size)

train_loader_distilbert = DataLoader(train_dataset_distilbert, batch_size=batch_size, shuffle=True)
dev_loader_distilbert = DataLoader(dev_dataset_distilbert, batch_size=batch_size)
test_loader_distilbert = DataLoader(test_dataset_distilbert, batch_size=batch_size)

2. Model Training and Evaluation

In [ ]:
# training function
def train(model, train_loader, dev_loader, epochs=3):
    optimizer = AdamW(model.parameters(), lr=2e-5) #5e-5 for bert and distilbert, #2e-5 for roberta, thus #2e-5 for all
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    scaler = GradScaler()

    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
            optimizer.zero_grad()

            with autocast():
                outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                loss = outputs.loss

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f}')

        # Evaluation on the validation set
        model.eval()
        preds, true_labels = [], []
        for batch in dev_loader:
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                outputs = model(b_input_ids, attention_mask=b_input_mask)
                logits = outputs.logits

            preds.extend(torch.argmax(logits, dim=1).tolist())
            true_labels.extend(b_labels.tolist())

        accuracy = accuracy_score(true_labels, preds)
        print(f'Epoch {epoch+1}/{epochs} - Validation Accuracy: {accuracy:.4f}')

In [ ]:
# Initialize and train models
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

print("Training BERT model")
train(bert_model, train_loader_bert, dev_loader_bert)

print("Training RoBERTa model")
train(roberta_model, train_loader_roberta, dev_loader_roberta)

print("Training DistilBERT model")
train(distilbert_model, train_loader_distilbert, dev_loader_distilbert)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

Epoch 1/3 - Loss: 0.7058
Epoch 1/3 - Validation Accuracy: 0.7984


<ipython-input-11-a45742e142a4>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 2/3 - Loss: 0.4654
Epoch 2/3 - Validation Accuracy: 0.8076


<ipython-input-11-a45742e142a4>:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 3/3 - Loss: 0.3427
Epoch 3/3 - Validation Accuracy: 0.8056


In [ ]:
# Evaluation function
def evaluate(model, test_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    model.eval()

    preds, true_labels = [], []
    for batch in test_loader:
        b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            outputs = model(b_input_ids, attention_mask=b_input_mask)
            logits = outputs.logits

        preds.extend(torch.argmax(logits, dim=1).tolist())
        true_labels.extend(b_labels.tolist())

    accuracy = accuracy_score(true_labels, preds)
    return accuracy

In [ ]:
# Evaluate the models
bert_accuracy = evaluate(bert_model, test_loader_bert)
roberta_accuracy = evaluate(roberta_model, test_loader_roberta)
distilbert_accuracy = evaluate(distilbert_model, test_loader_distilbert)

print(f'BERT Test Accuracy: {bert_accuracy:.4f}')
print(f'RoBERTa Test Accuracy: {roberta_accuracy:.4f}')
print(f'DistilBERT Test Accuracy: {distilbert_accuracy:.4f}')

BERT Test Accuracy: 0.8292
RoBERTa Test Accuracy: 0.8768
DistilBERT Test Accuracy: 0.8052


**II. PRE-TRAINED MODELS ON ENRICHED SNLI DATA**

1. Data Enrichment

In [ ]:
# Function to extract important words and their POS tags
def extract_important_words_with_pos(sentence):
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    important_words = [(word, pos) for word, pos in pos_tags
                       if pos.startswith(('NN', 'VB', 'JJ')) and word.lower() not in stop_words]
    return important_words

# Function to create word pairs by matching POS
def create_word_pairs_by_pos(words1, words2):
    pairs = []
    nouns1 = [word for word, pos in words1 if pos.startswith('NN')]
    nouns2 = [word for word, pos in words2 if pos.startswith('NN')]
    verbs1 = [word for word, pos in words1 if pos.startswith('VB')]
    verbs2 = [word for word, pos in words2 if pos.startswith('VB')]
    adjs1 = [word for word, pos in words1 if pos.startswith('JJ')]
    adjs2 = [word for word, pos in words2 if pos.startswith('JJ')]

    pairs.extend([(noun1, noun2) for noun1 in nouns1 for noun2 in nouns2])
    pairs.extend([(verb1, verb2) for verb1 in verbs1 for verb2 in verbs2])
    pairs.extend([(adj1, adj2) for adj1 in adjs1 for adj2 in adjs2])

    return pairs

def add_word_pairs_column(df, premise_col='premise', hypothesis_col='hypothesis'):
    df['pair'] = df.apply(lambda row: create_word_pairs_by_pos(
        extract_important_words_with_pos(row[premise_col]),
        extract_important_words_with_pos(row[hypothesis_col])
    ), axis=1)

    return df



In [ ]:
snli_train = add_word_pairs_column(snli_train)
snli_dev = add_word_pairs_column(snli_dev)
snli_test = add_word_pairs_column(snli_test)

In [ ]:
def enrich_relations(df1, df2):
    # Check if 'pair' column exists in df2
    if 'pair' not in df2.columns:
        raise KeyError("The 'pair' column is missing in df2. Make sure df2 contains this column.")

    # Make a copy of df1 and flatten the 'pair' column in df2
    df1 = df1.copy()
    df2 = df2.copy()
    df2['pair_flat'] = df2['pair'].apply(lambda x: [item for sublist in x for item in sublist])

    # Initialize an empty list to store the results
    result = []

    # Iterate through each row in df2
    for _, row in df2.iterrows():
        relations = []
        for pair in row['pair_flat']:
            # Check if the pair exists in df1
            match = df1[df1['pair'].apply(lambda x: pair in x)]
            if not match.empty:
                # Extend relations while keeping unique values
                relations.extend(match['relations'].tolist())

        # Remove duplicates from relations before creating a list
        unique_relations = set()
        for relation_list in relations:
            # Split the relations and convert each to lowercase
            for relation in relation_list.split(','):
                unique_relations.add(relation.strip().lower())

        # Create the output row with relations as a list
        result.append({
            'premise': row['premise'],
            'hypothesis': row['hypothesis'],
            'label': row['label'],
            'pair': row['pair'],
            'relations': list(unique_relations)
        })

    # Create the final DataFrame
    final_df = pd.DataFrame(result)
    final_df = final_df.drop(columns=['pair'])

    # Explode and create dummies for the 'relations' column
    df_exploded = final_df.explode("relations").drop_duplicates(keep="first")
    df_exploded = pd.concat([df_exploded, pd.get_dummies(df_exploded['relations']).astype("int8")], axis=1)

    # Group by and aggregate
    enriched_df = df_exploded.groupby(['premise', 'hypothesis', 'label'], as_index=False).agg({
        "relations": lambda x: sorted(set(x)),
        **{col: "max" for col in df_exploded.columns if col not in ['premise', 'hypothesis', 'label']}
    })

    return enriched_df


In [ ]:
enriched_snli_train = enrich_relations(train_relation, snli_train)
enriched_snli_dev = enrich_relations(train_relation, snli_dev)
enriched_snli_test = enrich_relations(train_relation, snli_test)

In [ ]:
# Drop the relations column
def drop_relation_column(train_df, dev_df, test_df):
    train_df = train_df.drop(columns=['relations'], errors='ignore')
    dev_df = dev_df.drop(columns=['relations'], errors='ignore')
    test_df = test_df.drop(columns=['relations'], errors='ignore')
    return train_df, dev_df, test_df

enriched_snli_train, enriched_snli_dev, enriched_snli_test = drop_relation_column(
    enriched_snli_train, enriched_snli_dev, enriched_snli_test
)

2. Model Training

In [ ]:
# Pre-tokenizing dataset for efficiency
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.input_ids = []
        self.attention_masks = []
        self.features = torch.tensor(dataframe[["antonym", "atlocation", "capableof", "causes", "causesdesire",
                                                 "createdby", "definedas", "derivedfrom", "formof", "isa"]].values, dtype=torch.float)
        self.labels = torch.tensor(dataframe["label"].values, dtype=torch.long)

        for _, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Tokenizing"):
            encoded = tokenizer(
                row["premise"] + " " + row["hypothesis"],
                truncation=True,
                padding="max_length",
                max_length=max_length,
                return_tensors="pt"
            )
            self.input_ids.append(encoded["input_ids"].squeeze())
            self.attention_masks.append(encoded["attention_mask"].squeeze())

        self.input_ids = torch.stack(self.input_ids)
        self.attention_masks = torch.stack(self.attention_masks)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        return {
            "input_ids": self.input_ids[index],
            "attention_mask": self.attention_masks[index],
            "features": self.features[index],
            "label": self.labels[index],
        }

class CustomModel(nn.Module):
    def __init__(self, model_name, num_labels, feature_dim):
        super(CustomModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.feature_layer = nn.Linear(feature_dim, 16)
        self.dropout = nn.Dropout(0.3)

        if "distilbert" in model_name:
            self.use_pooler = False
            self.classifier = nn.Linear(self.bert.config.hidden_size + 16, num_labels)
        else:
            self.use_pooler = True
            self.classifier = nn.Linear(self.bert.config.hidden_size + 16, num_labels)

    def forward(self, input_ids, attention_mask, features):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)

        if self.use_pooler:
            pooled_output = bert_output.pooler_output
        else:
            pooled_output = torch.mean(bert_output.last_hidden_state, dim=1)

        feature_output = self.dropout(nn.ReLU()(self.feature_layer(features)))
        combined_output = torch.cat((pooled_output, feature_output), dim=1)
        logits = self.classifier(combined_output)
        return logits

# Training function with mixed precision
def train(model, dataloader, optimizer, loss_fn, scaler, device, accumulation_steps=2):
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(dataloader, desc="Training", leave=False)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        features = batch["features"].to(device)
        labels = batch["label"].to(device)

        with autocast():
            logits = model(input_ids, attention_mask, features)
            loss = loss_fn(logits, labels) / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * accumulation_steps

    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    all_labels, all_preds = [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            features = batch["features"].to(device)
            labels = batch["label"].to(device)

            logits = model(input_ids, attention_mask, features)
            preds = torch.argmax(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Main function
if __name__ == "__main__":
    train_df = enriched_snli_train
    dev_df = enriched_snli_dev
    test_df = enriched_snli_test

    model_names = {
        "BERT": "bert-base-uncased",
        "RoBERTa": "roberta-base",
        "DistilBERT": "distilbert-base-uncased"
    }
    num_labels = 3
    feature_dim = 10
    max_length = 128
    batch_size = 16
    num_epochs = 3
    learning_rate = 2e-5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    accumulation_steps = 2
    num_workers = 4 if torch.cuda.is_available() else 0

    for model_key, model_name in model_names.items():
        print(f"\nTraining {model_key} model...")

        tokenizer = AutoTokenizer.from_pretrained(model_name)
        train_dataset = CustomDataset(train_df, tokenizer, max_length)
        dev_dataset = CustomDataset(dev_df, tokenizer, max_length)
        test_dataset = CustomDataset(test_df, tokenizer, max_length)

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
        dev_loader = DataLoader(dev_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True)

        model = CustomModel(model_name, num_labels, feature_dim).to(device)
        optimizer = AdamW(model.parameters(), lr=learning_rate)
        loss_fn = nn.CrossEntropyLoss()
        scaler = GradScaler()

        for epoch in range(num_epochs):
            print(f"Epoch {epoch + 1}/{num_epochs}")
            train_loss = train(model, train_loader, optimizer, loss_fn, scaler, device, accumulation_steps)
            dev_metrics = evaluate(model, dev_loader, device)

            print(f"Train Loss: {train_loss:.4f}")
            print(f"Dev Metrics: {dev_metrics}")

        test_metrics = evaluate(model, test_loader, device)
        print(f"Test Metrics: {test_metrics}")

        torch.save(model.state_dict(), f"{model_key}_custom_model.pth")



Training BERT model...


Tokenizing: 100%|██████████| 2500/2500 [00:01<00:00, 2200.58it/s]
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-20-f38520fb6ada>:147: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


Epoch 1/3


Training:   0%|          | 0/1249 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-20-f38520fb6ada>:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.6919
Dev Metrics: {'accuracy': 0.8256, 'precision': 0.8261523986714743, 'recall': 0.8256, 'f1': 0.824520231018772}
Epoch 2/3


...

Train Loss: 0.4829
Dev Metrics: {'accuracy': 0.8044, 'precision': 0.8065911217990519, 'recall': 0.8044, 'f1': 0.8023987441682439}
Epoch 3/3


Training:   0%|          | 0/1249 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
<ipython-input-20-f38520fb6ada>:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train Loss: 0.3416
Dev Metrics: {'accuracy': 0.8128, 'precision': 0.8121438015980933, 'recall': 0.8128, 'f1': 0.8121621718605746}


Evaluating:   0%|          | 0/157 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Test Metrics: {'accuracy': 0.7988, 'precision': 0.7992620376849394, 'recall': 0.7988, 'f1': 0.798754076534988}
